# Немного кода для обкачивания интернета
## Модуль [requests](https://realpython.com/python-requests/)

Давайте посмотрим html формат

Лучший учебник - https://www.w3schools.com

![](https://raw.githubusercontent.com/tbkazakova/compling_for_lyceum/main/data/tag.png "tag")

Вот пример очень простого сайта в интернете https://itcorp.com/, зарегистрированного в 1986 году.

И сайта посложнее https://spork.org/, посвящённый ложковилке (последнее обновление в 1996).

Теперь скачаем html страничку

In [38]:
import requests

In [39]:
response = requests.get("https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/kitayskiy-novyy-god-v-moskovskom-zooparke/", verify=False)

In [40]:
# отключим предупреждения
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
disable_warnings(InsecureRequestWarning)

response = requests.get("https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/kitayskiy-novyy-god-v-moskovskom-zooparke/", verify=False)

В `response` теперь лежит ответ сервера. Это не просто html-код страницы, а еще дополнительная информация

In [41]:
response

<Response [200]>

In [42]:
response.status_code
# 200 - "всё работает"
# 404, 503 и др. - "произошла ошибка, не работает, такой страницы нет и т.д."

200

In [43]:
# Вывели первые 210 символов html
print(response.text[:210])

<!doctype html>
<html lang="ru">
<head>
	<meta name="yandex-verification" content="60483b355f0f5c1d" />
	<link rel="icon" type="image/x-icon" href="/local/templates/zoo/favicon.ico" />
		<meta http-equiv="Conte


### Что делать, если сайт защищается?

Если он защищается от краулеров, можно, например, представиться Мозиллой:

In [44]:
url = 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/kitayskiy-novyy-god-v-moskovskom-zooparke/'
# адрес страницы, которую мы хотим скачать
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

response = requests.get(url, headers={'User-Agent':user_agent}, verify=False)


Или использовать **специальную библиотеку**:

In [ ]:
! pip3 install fake-useragent

In [46]:
from fake_useragent import UserAgent

In [47]:
user_agent = UserAgent().chrome
user_agent

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'

In [48]:
response = requests.get(url,
                        headers={'User-Agent':user_agent},
                        verify=False)

Или **прокси-сервер**. Это дополнительное звено между вами и интернетом, через него пойдет подключение и сайт не будет знать, что это вы посылаете запрос.

Адреса прокси можно взять со специальных сайтов. И потом проверить, что они рабочие, прежде чем использовать https://checkerproxy.net/

[Пример сервиса](https://whatismyipaddress.com/ip-lookup), который позволяет с некоторой точностью отследить по IP адресу

Возвращаемся к скачиванию странички.

In [49]:
response.url  #ссылка

'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/kitayskiy-novyy-god-v-moskovskom-zooparke/'

In [50]:
print(response.text[:300])  #html код

<!doctype html>
<html lang="ru">
<head>
	<meta name="yandex-verification" content="60483b355f0f5c1d" />
	<link rel="icon" type="image/x-icon" href="/local/templates/zoo/favicon.ico" />
		<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<script type="text/javascript" data-skip-mo


Существует несколько вариантов, как достать что-то из определенного тега, например, достать заголовок:

- регулярные выражения (плохой вариант)
- специальные библиотеки питона, например, BeautifulSoup (bs4) или lxml (хороший вариант)


Регулярными выражениями мы уже умеем:

In [51]:
import re

In [52]:
code = response.text
re.search(r'<div class="dp-content-inner" style="color:#fff;">(.|\n)*?<\/div>', code).group()

'<div class="dp-content-inner" style="color:#fff;">\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\n<div class="dp-content border-block">\n\t<div class="container">\n\t\t<div class="row">\n\t\t\t<div class="dp-content-inner" style="color:#fff;">\n\t\t\t\t\t\t\t\t\t\t\t\t<p>\r\n\t С 9 по 18 февраля в столице пройдет фестиваль «Китайский Новый год в Москве». Московский зоопарк стал одной из площадок для празднования и подготовил ряд тематических мероприятий: экскурсии, лекции и мастер-классы. А 9, 10 и 11 февраля попасть в зоопарк можно будет бесплатно, переодевшись в костюм панды. Такую же возможность предоставят посетительницам с именем Екатерина – тезкам малышки-панды Катюши.\r\n</p>\r\n<p>\r\n\t 9 февраля в 11:00 в открытом лектории Московского зоопарка пройдет лекция «Большие панды - загадочные гости с Востока». Посещение бесплатно по предварительной записи: <a href="https://moscowzoo.ru/education/public-lectures/">https://moscowzoo.ru/education/public-lectures/</a>\r\n</p>\r\n<p>\r\n\t 10 и 11 фе

Теперь умный способ:

In [53]:
from bs4 import BeautifulSoup

In [54]:
# инициализируем (создаем) soup из response.text (кода страницы)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify()[:500])  # печатаем приукрашенный суп

<!DOCTYPE html>
<html lang="ru">
 <head>
  <meta content="60483b355f0f5c1d" name="yandex-verification"/>
  <link href="/local/templates/zoo/favicon.ico" rel="icon" type="image/x-icon"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script data-skip-moving="true" type="text/javascript">
   (function(w, d, n) {var cl = "bx-core";var ht = d.documentElement;var htc = ht ? ht.className : undefined;if (htc === undefined || htc.indexOf(cl) !== -1){return;}var ua = n.userAgen


Если мы хотим вывести заголовок, нужно посмотреть, какими тегами он оформляется.

Заходим на наш сайт

На этом конкретном сайте заголовки лежат внутри тегов `cp-banner`

In [55]:
name = soup.find('cp-banner')
print(name.prettify())

<cp-banner>
 <template #title="">
  Китайский Новый год в Московском зоопарке
 </template>
 <template #description="">
 </template>
</cp-banner>



А внутри этого раздела текст заголовка лежит внутри первого тега `template`

In [56]:
name2 = name.find('template')
print(name2.prettify())

<template #title="">
 Китайский Новый год в Московском зоопарке
</template>



Нас интересует только текст, поэтому сделаем `get_text`

In [57]:
print(name2.get_text())

Китайский Новый год в Московском зоопарке


\- Почему мы не сделали сразу `soup.find('template')`?

\- Потому что тегов `template` несколько, а мы ищем уникальное обрамление нашего заголовка.

In [59]:
# Проверьте, что будет, если искать по "template"
# soup.find('template')

In [60]:
# Вот так мы попросили найти все теги template внутри cp-banner
soup.select('cp-banner > template')

[<template #title="">Китайский Новый год в Московском зоопарке</template>,
 <template #description=""></template>]

In [61]:
# Если мы уверены, что заголовок в первом из найденных тегов template,
# можем вывести первый тег
print(soup.select('cp-banner > template')[0])

# и его содержание
print('Содержание тега:')
print(soup.select('cp-banner > template')[0].get_text())

<template #title="">Китайский Новый год в Московском зоопарке</template>
Содержание тега:
Китайский Новый год в Московском зоопарке


А теперь найдём текст новости.

Он лежит в теге `<div class="dp-content-inner" style="color:#fff;">`

\- Как устроен этот тег?

\- div - имя тега, class и style - имена атрибутов, "dp-content-inner" и "color:#fff;" - значения атрибутов

In [63]:
post = soup.find('div', {'class':"dp-content-inner", "style":"color:#fff;"})  # вот так можно указать атрибуты тегов
print(post.prettify())

<div class="dp-content-inner" style="color:#fff;">
 <div class="dp-content border-block">
  <div class="container">
   <div class="row">
    <div class="dp-content-inner" style="color:#fff;">
     <p>
      С 9 по 18 февраля в столице пройдет фестиваль «Китайский Новый год в Москве». Московский зоопарк стал одной из площадок для празднования и подготовил ряд тематических мероприятий: экскурсии, лекции и мастер-классы. А 9, 10 и 11 февраля попасть в зоопарк можно будет бесплатно, переодевшись в костюм панды. Такую же возможность предоставят посетительницам с именем Екатерина – тезкам малышки-панды Катюши.
     </p>
     <p>
      9 февраля в 11:00 в открытом лектории Московского зоопарка пройдет лекция «Большие панды - загадочные гости с Востока». Посещение бесплатно по предварительной записи:
      <a href="https://moscowzoo.ru/education/public-lectures/">
       https://moscowzoo.ru/education/public-lectures/
      </a>
     </p>
     <p>
      10 и 11 февраля в 12:00, а также 22, 23 

In [64]:
text = post.get_text()  # получили текст
print(text)







	 С 9 по 18 февраля в столице пройдет фестиваль «Китайский Новый год в Москве». Московский зоопарк стал одной из площадок для празднования и подготовил ряд тематических мероприятий: экскурсии, лекции и мастер-классы. А 9, 10 и 11 февраля попасть в зоопарк можно будет бесплатно, переодевшись в костюм панды. Такую же возможность предоставят посетительницам с именем Екатерина – тезкам малышки-панды Катюши.


	 9 февраля в 11:00 в открытом лектории Московского зоопарка пройдет лекция «Большие панды - загадочные гости с Востока». Посещение бесплатно по предварительной записи: https://moscowzoo.ru/education/public-lectures/


	 10 и 11 февраля в 12:00, а также 22, 23 и 24 февраля в 14:00 пройдет экскурсия «Гости из Поднебесной». Это семейная образовательная программа, посвященная фауне Китая: большим пандам, красным пандам и гималайским медведям, а также природоохранным программам зоопарка. Запись на экскурсию «Гости из Поднебесной» по электронной почте: education@moscowzoo.ru


	 17 ф

In [65]:
text

'\n\n\n\n\n\r\n\t С 9 по 18 февраля в столице пройдет фестиваль «Китайский Новый год в Москве». Московский зоопарк стал одной из площадок для празднования и подготовил ряд тематических мероприятий: экскурсии, лекции и мастер-классы. А 9, 10 и 11 февраля попасть в зоопарк можно будет бесплатно, переодевшись в костюм панды. Такую же возможность предоставят посетительницам с именем Екатерина – тезкам малышки-панды Катюши.\r\n\n\r\n\t 9 февраля в 11:00 в открытом лектории Московского зоопарка пройдет лекция «Большие панды - загадочные гости с Востока». Посещение бесплатно по предварительной записи: https://moscowzoo.ru/education/public-lectures/\n\n\r\n\t 10 и 11 февраля в 12:00, а также 22, 23 и 24 февраля в 14:00 пройдет экскурсия «Гости из Поднебесной». Это семейная образовательная программа, посвященная фауне Китая: большим пандам, красным пандам и гималайским медведям, а также природоохранным программам зоопарка. Запись на экскурсию «Гости из Поднебесной» по электронной почте: educati

Вот здесь можно [посмотреть часть документации BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html)

### Пишем краулер

Теперь давайте обкачаем несколько новостей.

1. Где взять ссылки на новости?

In [22]:
news_url = 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/'
response = requests.get(news_url,
                        headers={'User-Agent':user_agent},
                        verify=False)
soup = BeautifulSoup(response.text, 'html.parser')

In [23]:
soup.find_all('a', {'class':"article-item-link"})

[<a class="article-item-link" href="/about-zoo/news/novosti-zooparka/kitayskiy-novyy-god-v-moskovskom-zooparke/">Читать новость</a>,
 <a class="article-item-link" href="/about-zoo/news/novosti-zooparka/30-yanvarya-zakryta-vystavka-udivitelnyy-mir-reptiliy/">Читать новость</a>,
 <a class="article-item-link" href="/about-zoo/news/novosti-zooparka/segodnya-v-moskovskom-zooparke-den-pamyati-vladimira-vladimirovicha-spitsina/">Читать новость</a>,
 <a class="article-item-link" href="/about-zoo/news/novosti-zooparka/remont-perekhodnogo-mosta/">Читать новость</a>]

2. Как вытащить ссылки из тегов?

In [24]:
for a in soup.find_all('a', {'class':"article-item-link"}):
    print(a['href'])

/about-zoo/news/novosti-zooparka/kitayskiy-novyy-god-v-moskovskom-zooparke/
/about-zoo/news/novosti-zooparka/30-yanvarya-zakryta-vystavka-udivitelnyy-mir-reptiliy/
/about-zoo/news/novosti-zooparka/segodnya-v-moskovskom-zooparke-den-pamyati-vladimira-vladimirovicha-spitsina/
/about-zoo/news/novosti-zooparka/remont-perekhodnogo-mosta/


3. Как ходить по страницам списков новостей, чтобы доставать ссылки?

In [25]:
news_urls = []
news_url = 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/?PAGEN_1='
for i in range(1, 6):
    response = requests.get(news_url+str(i),
                        headers={'User-Agent':user_agent},
                        verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    for a in soup.find_all('a', {'class':"article-item-link"}):
        news_urls.append('https://www.moscowzoo.ru'+a['href'])
news_urls

['https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/kitayskiy-novyy-god-v-moskovskom-zooparke/',
 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/30-yanvarya-zakryta-vystavka-udivitelnyy-mir-reptiliy/',
 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/segodnya-v-moskovskom-zooparke-den-pamyati-vladimira-vladimirovicha-spitsina/',
 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/remont-perekhodnogo-mosta/',
 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/v-moskovskom-zooparke-otkrylas-vystavka-zveri-ptitsy-i-illyuzii-vladimira-stakheeva/',
 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/kanikuly-s-kipoy-i-pushey-vozvrashchayutsya-/',
 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/15-dekabrya-v-moskovskom-zooparka-poyavilis-na-svet-dva-detenysha-kapibary/',
 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/zborelok2024/',
 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/moskovskiy-zoopark-okazal-sro

In [26]:
# вытащим тексты этих новостей
news_texts = []
for link in news_urls:
    response = requests.get(link,
                        headers={'User-Agent':user_agent},
                        verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    post = soup.find('div', {'class':"dp-content-inner", "style":"color:#fff;"}).get_text()
    news_texts.append(post.strip())

In [27]:
news_texts[7]

'Московский зоопарк запускает традиционную новогоднюю акцию по сбору ёлок, сосен и пихт. Хвойные деревья украсят вольеры наших обитателей, станут для них новой игрушкой или полезным лакомством.\r\n\r\n«Как и всегда, передать елку, пихту или сосну в дар могут только компании. От частных лиц деревья не принимаются. Елки, которые вы украшали дома, могут быть опасны для животных. На их ветках нередко остаются фрагменты пластиковых или стеклянных украшений, мишуры, гирлянд, искусственного снега. Кроме того, деревья, которые долго простояли в помещении, теряют свой аромат и становятся не очень интересными для животных», - рассказала генеральный директор Московского зоопарка Светлана Акулова.\r\n\r\nИграть с пушистыми деревьями любят практически все обитател зоопарка: панды, крупные кошки и маленькие обитатели Фауны России. Полакомиться еловыми ветками не против винторогие козлы, голубые бараны, азиатские слоны, альпаки и другие. Для них ёлки становятся пищевым обогащением среды обитания. Хво

In [29]:
len(news_texts)

20

## NER. Дубль 2
### (Named-entity recognition, извлечение именованных сущностей)

Именнованные сущности: персоны, локации, организации (а также временные выражения, валюты, медицинские коды и др.)

**Как обычно размечают последовательности токенов?**

BIOES-схема: к метке сущности (например, PER для персон или ORG для организаций) добавляется префикс, который обозначает позицию токена в спане сущности:

B – beginning – первый токен в спане сущности, которая состоит из нескольких токенов;

I – inside – внутри спана;

О – outside – токен не относится ни к какой сущности;

E – ending – последний токен сущности, которая состоит из нескольких токенов;

S – single – сущность состоит из одного токена.

До какого количества префиксов можно сократить?

In [ ]:
!pip3 install -U pip setuptools wheel
!pip3 install -U spacy

In [ ]:
! python3 -m spacy download ru_core_news_sm

In [33]:
import spacy

In [34]:
nlp = spacy.load("ru_core_news_sm")

In [35]:
url = 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/pervyy-v-rossii-detenysh-bolshoy-pandy/'
response = requests.get(url,
                        headers={'User-Agent':user_agent},
                        verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
post = soup.find('div', {'class':"dp-content-inner", "style":"color:#fff;"}).get_text()
text = post.strip()
text

'В Московском зоопарке появился на свет первый в истории России детеныш большой панды. Это уникальное событие как для страны, так и для всего мирового природоохранного сообщества. Панды переданы Московскому зоопарку в рамках международной программы по сохранению, защите и исследованию больших панд. Миссия зоопарка – внести значимый вклад в создание резервной популяции этих редчайших животных.\xa0\r\n \xa0\r\n Малыш весом около 150 граммов родился у молодой пары больших панд — самца Жуи и самки Диндин. Пол детеныша пока неизвестен. Наша Диндин проявила большую заботу и сразу приняла малыша. За самкой круглосуточно наблюдают и берут все необходимые анализы.\xa0\r\n \xa0\r\n Первый в России детеныш большой панды появился на свет после спаривания естественным путём. Чтобы помочь коллегам, в Москву на три месяца прибыл эксперт по размножению больших панд из Исследовательского центра по сохранению больших панд Китая. Специалисты не торопили животных, четко следовали указаниям китайских колле

In [36]:
doc = nlp(text.strip())
doc

В Московском зоопарке появился на свет первый в истории России детеныш большой панды. Это уникальное событие как для страны, так и для всего мирового природоохранного сообщества. Панды переданы Московскому зоопарку в рамках международной программы по сохранению, защите и исследованию больших панд. Миссия зоопарка – внести значимый вклад в создание резервной популяции этих редчайших животных. 
  
 Малыш весом около 150 граммов родился у молодой пары больших панд — самца Жуи и самки Диндин. Пол детеныша пока неизвестен. Наша Диндин проявила большую заботу и сразу приняла малыша. За самкой круглосуточно наблюдают и берут все необходимые анализы. 
  
 Первый в России детеныш большой панды появился на свет после спаривания естественным путём. Чтобы помочь коллегам, в Москву на три месяца прибыл эксперт по размножению больших панд из Исследовательского центра по сохранению больших панд Китая. Специалисты не торопили животных, четко следовали указаниям китайских коллег, добивались идеальных у

In [37]:
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Московском зоопарке 2 21 ORG
России 56 62 LOC
Московскому зоопарку 194 214 LOC
Диндин 488 494 PER
Диндин 531 537 PER
России 669 675 LOC
Москву 777 783 LOC
Исследовательского центра по сохранению больших панд 844 896 ORG
Китая 897 902 LOC
Жуи 1114 1117 PER
Диндин 1120 1126 PER
Москву 1429 1435 LOC
Пекина 1478 1484 LOC
Фауна Китая 1519 1530 ORG
Московском зоопарке 1534 1553 ORG
России 1579 1585 LOC
Владимир Путин 1586 1600 PER
Китайской Народной Республики 1616 1645 LOC
Си Цзиньпин 1646 1657 PER
